# Dynamic Control using Traefik

This notebook will show how we can use the Traefik proxy to deal with incoming traffic

In [1]:
import syft as sy
from syft.core.node.new.client import SyftClient

In [2]:
w = sy.Worker()

> Starting Worker: Quirky Hochreiter - eea1e3b4bd1b4071b7bffb2edc7b8c08 - NodeType.DOMAIN - [<class 'syft.core.node.new.user_service.UserService'>, <class 'syft.core.node.new.metadata_service.MetadataService'>, <class 'syft.core.node.new.action_service.ActionService'>, <class 'syft.core.node.new.test_service.TestService'>, <class 'syft.core.node.new.dataset_service.DatasetService'>, <class 'syft.core.node.new.user_code_service.UserCodeService'>, <class 'syft.core.node.new.request_service.RequestService'>, <class 'syft.core.node.new.data_subject_service.DataSubjectService'>, <class 'syft.core.node.new.network_service.NetworkService'>, <class 'syft.core.node.new.policy_service.PolicyService'>, <class 'syft.core.node.new.message_service.MessageService'>, <class 'syft.core.node.new.project_service.ProjectService'>, <class 'syft.core.node.new.data_subject_member_service.DataSubjectMemberService'>]


In [3]:
client = SyftClient.from_node(w).login(email="info@openmined.org", password="changethis")

### Blocking Traffic from an IP address or hostname

- with logging
- with persistence even after domain node restarts

In [4]:
client.datasets

In [5]:
from syft.core.node.new.action_object import ActionObject

In [7]:
import numpy as np
obj = ActionObject.from_obj(np.random.rand(5,5))

In [8]:
ptr = obj.send(client)

In [9]:
ptr

array([[0.74074244, 0.5196783 , 0.27361163, 0.35009409, 0.93692262],
       [0.84284931, 0.79161962, 0.21222134, 0.71800114, 0.50034856],
       [0.8587059 , 0.86647233, 0.94541335, 0.05943639, 0.07130845],
       [0.38826281, 0.28790051, 0.89910204, 0.97376507, 0.1253996 ],
       [0.36721699, 0.64349734, 0.57207201, 0.23548878, 0.15646151]])

In [10]:
domain = sy.login(email="info@openmined.org", password="changethis", port=8081)

Logged into Priceless Gardner as <info@openmined.org>


In [11]:
domain.datasets

In [14]:
guest_client = domain.peer.guest_client

In [15]:
guest_client

<SyftClient - Priceless Gardner <09cbf124ee7d43d2b11282f07b8a0712>: HTTPConnection: http://localhost:8081>

In [17]:
ptr2 = obj.send(guest_client)

In [18]:
ptr2 + ptr

array([[1.48148488, 1.03935659, 0.54722327, 0.70018818, 1.87384524],
       [1.68569863, 1.58323924, 0.42444269, 1.43600227, 1.00069711],
       [1.7174118 , 1.73294467, 1.8908267 , 0.11887277, 0.1426169 ],
       [0.77652563, 0.57580101, 1.79820407, 1.94753014, 0.2507992 ],
       [0.73443398, 1.28699469, 1.14414403, 0.47097756, 0.31292302]])